In [1]:
#!pip install statsmodels

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

In [16]:
df = pd.read_csv("homework_2.1.csv")

In [17]:
df.head()

,time,G1,G2,G3
0,0,0.882026,1.441575,0.065409
1,1,0.210079,-0.163880,0.140310
2,2,0.509369,-0.115242,0.819830
3,3,1.150447,1.014698,0.607632
4,4,0.973779,-0.046562,0.610066


For questions 1 and 2: 

Do a regression to estimate the fixed effect of each group. We assume that there is one single linear coefficient for all the data, plus the fixed effect of each group. Use the file homework_2.1.csv.  The variables G1, G2, and G3 are the outcomes and the time is the treatment.

Question 1
Which of these is closest to being the coefficient of group 1? 

Option A
0.00850

Option B
0.00485

Option C
0.1823 

Option D
0.01023 



In [18]:
# Use only Group 1 data
X = df["time"]
Y = df["G1"]

# Add constant (intercept)
X_with_const = sm.add_constant(X)

# Fit OLS
model = sm.OLS(Y, X_with_const).fit()

# Show results
print(model.params)

const    0.104236
time     0.008498
dtype: float64


In [20]:
import pandas as pd
from sklearn.linear_model import LinearRegression


# Extract time and G1 only
X = df[["time"]]   # make sure it's 2D
y = df["G1"]

# Fit the model
model = LinearRegression()
model.fit(X, y)

# Print intercept and slope (coefficient on time)
print("Intercept (Group 1 fixed effect):", model.intercept_)
print("Coefficient on time, slope:", model.coef_[0])


Intercept (Group 1 fixed effect): 0.104235723914613
Coefficient on time, slope: 0.008498349168750093


Which of these is closest to being the common linear coefficient for all groups?

Option A
0.01120 

Option B
0.01003

Option C
0.009017 

Option D
0.2808 

In [22]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression


# Step 1: Stack the outcomes into a single Y array
Y = np.hstack([df["G1"].values, df["G2"].values, df["G3"].values])

# Step 2: Create group labels: 1 for G1, 2 for G2, 3 for G3
groups = np.repeat([1, 2, 3], len(df))

# Step 3: Create the repeated time variable (X)
X_time = np.tile(df["time"].values, 3).reshape(-1, 1)

# Step 4: Create dummy variables for fixed effects (G2, G3 only)
group_dummies = pd.get_dummies(groups, drop_first=True)
group_dummies.columns = ["G2", "G3"]

# Step 5: Combine time and group fixed effects
X_design = np.hstack([X_time, group_dummies.values])

# Step 6: Fit model
model = LinearRegression()
model.fit(X_design, Y)

# Step 7: Get coefficient on time (this is the common slope)
common_beta = model.coef_[0]
print("Common linear coefficient (slope on time):", common_beta)


Common linear coefficient (slope on time): 0.009017213376688272


In [23]:


# Step 1: Stack Y values from G1, G2, G3
Y = np.hstack([df["G1"].values, df["G2"].values, df["G3"].values])

# Step 2: Create group labels
groups = np.repeat([1, 2, 3], len(df))

# Step 3: Create time variable repeated 3 times
X = np.tile(df["time"].values, 3).reshape(-1, 1)

# Step 4: Create fixed effect dummies (G2 and G3 only)
X_fixed_effects = pd.get_dummies(groups, drop_first=True) * 1  # G2, G3

# Step 5: Combine time + group dummies into design matrix
df_design = pd.DataFrame(
    np.hstack((X, X_fixed_effects)),
    columns=["X", "G2", "G3"]
)

# Step 6: Run OLS
results_fixed_effects = sm.OLS(Y, sm.add_constant(df_design)).fit()

# Step 7: Print results
print(results_fixed_effects.params)


const    0.078552
X        0.009017
G2       0.511102
G3       0.190480
dtype: float64


For questions 3-5:

Given a data set, create a bootstrap simulation to try different possibilities. 
Use the file homework_2.2.csv 


Question 3:


If we were to measure the effect of the treatment simply by subtracting the outcomes of the treated vs. untreated population, which of these is closest to the mean effect? (This is not the recommended way of measuring the mean effect when there are confounders!) 

Option A
12.831

Option B
5.149

Option C
8.031

Option D
2.921



In [25]:
df2 = pd.read_csv("homework_2.2.csv")

In [26]:
# Split treated and untreated based on X
treated = df2[df2["X"] == 1]["Y"]
untreated = df2[df2["X"] == 0]["Y"]

# Compute naive difference in means
naive_effect = treated.mean() - untreated.mean()
print("Naive mean treatment effect:", naive_effect)

Naive mean treatment effect: 2.9207172647231907


Question 4:

If we were to use bootstrap sampling to measure the variance of that effect, again finding the effect using the non-recommended approach, which of these is closest to that variance?

0.001822

Option B
0.1280

Option C
0.03274 

Option D
0.002388 


In [29]:

# Number of bootstrap samples
n_bootstrap = 1000

# Store bootstrap estimates
bootstrap_effects = []

for _ in range(n_bootstrap):
    # Sample with replacement from the full dataset
    sample = df2.sample(frac=1, replace=True)

    # Naive effect in this bootstrap sample
    treated = sample[sample["X"] == 1]["Y"]
    untreated = sample[sample["X"] == 0]["Y"]

    # Defensive check in case a sample has no treated/untreated
    if len(treated) > 0 and len(untreated) > 0:
        effect = treated.mean() - untreated.mean()
        bootstrap_effects.append(effect)

# Compute variance of bootstrap estimates
bootstrap_variance = np.var(bootstrap_effects, ddof=1)
print("Bootstrap variance of naive treatment effect:", bootstrap_variance)


Bootstrap variance of naive treatment effect: 0.031566391496901665


In [28]:
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

# Run 10,000 bootstrap samples
n_bootstrap = 10_000
bootstrap_effects = []

for _ in range(n_bootstrap):
    # Resample the dataset with replacement
    sample = df2.sample(frac=1, replace=True)

    # Compute naive treatment effect in this sample
    treated = sample[sample["X"] == 1]["Y"]
    untreated = sample[sample["X"] == 0]["Y"]

    # Avoid division by 0 in rare cases
    if len(treated) > 0 and len(untreated) > 0:
        effect = treated.mean() - untreated.mean()
        bootstrap_effects.append(effect)

# Compute sample variance of the bootstrapped effects
bootstrap_variance = np.var(bootstrap_effects, ddof=1)
print("Bootstrap variance of naive treatment effect:", bootstrap_variance)


Bootstrap variance of naive treatment effect: 0.03255543091954204


In [30]:
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

# Run 10,000 bootstrap samples
n_bootstrap = 100_000
bootstrap_effects = []

for _ in range(n_bootstrap):
    # Resample the dataset with replacement
    sample = df2.sample(frac=1, replace=True)

    # Compute naive treatment effect in this sample
    treated = sample[sample["X"] == 1]["Y"]
    untreated = sample[sample["X"] == 0]["Y"]

    # Avoid division by 0 in rare cases
    if len(treated) > 0 and len(untreated) > 0:
        effect = treated.mean() - untreated.mean()
        bootstrap_effects.append(effect)

# Compute sample variance of the bootstrapped effects
bootstrap_variance = np.var(bootstrap_effects, ddof=1)
print("Bootstrap variance of naive treatment effect:", bootstrap_variance)


Bootstrap variance of naive treatment effect: 0.03210191650988605


Question 5

if we ran a linear regression (with intercept) to measure the effect, which of these is closest to the skewness of the effect measured? (Look up skewness online. You can use scipy.stats.skew to compute the skewness of a list of numbers.) 

Option A
0.2315

Option B
0.04850

Option C
0.09812

Option D
0.3223



In [32]:
from scipy.stats import skew

np.random.seed(42)
n_bootstrap = 10_000
boot_coef = []

for _ in range(n_bootstrap):
    # Bootstrap sample
    sample = df2.sample(frac=1, replace=True)

    # Define X and y for regression
    X_boot = sample[["X"]]  # Treatment indicator
    y_boot = sample["Y"]

    # Fit linear regression with intercept
    model = LinearRegression()
    model.fit(X_boot, y_boot)

    # Store the coefficient on treatment (X)
    boot_coef.append(model.coef_[0])

# Compute skewness of the bootstrapped treatment effects
skewness_value = skew(boot_coef)
print("Skewness of the treatment effect estimates:", skewness_value)


Skewness of the treatment effect estimates: 0.040745945844387385
